In [1]:
import pandas as pd
import pickle
import numpy as np
import time
import datetime

In [36]:
from math import radians, sin, cos, atan2, sqrt

In [2]:
a15 = pd.read_csv("./2015.csv")
a14 = pd.read_csv("./2014.csv")

/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
a14 = a14[["Longitude", "Latitude", "Date"]]
a15 = a15[["Longitude", "Latitude", "Date"]]

In [4]:
a14['loco'] = list(zip(a14.Longitude, a14.Latitude))
a15['loco'] = list(zip(a15.Longitude, a15.Latitude))

In [5]:
a14.head()

,Longitude,Latitude,Date,loco
0,-0.206443,51.496345,09/01/2014,"(-0.206443, 51.496345)"
1,-0.189713,51.489523,20/01/2014,"(-0.189713, 51.489523)"
2,-0.173827,51.490536,21/01/2014,"(-0.173827, 51.490536)"
3,-0.192311,51.496668,15/01/2014,"(-0.192311, 51.496668)"
4,-0.171308,51.495892,09/01/2014,"(-0.171308, 51.495892)"


In [6]:
with open("../bdc/hjt_map_dict.pickle", "r") as p:
    section_id_info = pickle.load(p)

In [24]:
idloc = pd.DataFrame(section_id_info).T.location

In [7]:
locs = []
for i in section_id_info:
    locs.append(section_id_info[i]["location"])

In [8]:
print np.matrix(locs)[:,0].min()
print np.matrix(locs)[:,0].max()
print np.matrix(locs)[:,1].min()
print np.matrix(locs)[:,1].max()

51.424
51.479
-2.629
-2.541


In [43]:
def inBrs(a):
    if -2.629 <= a[0] and a[0] <= -2.541 and 51.424 <= a[1] and a[1] <= 51.479 :
        return True
    else:
        return False
brs14 = a14[a14["loco"].map(inBrs)]
brs15 = a15[a15["loco"].map(inBrs)]

In [44]:
def d2s(d):
    d += " 12:00:00"
    return int(time.mktime(datetime.datetime.strptime(d, "%d/%m/%Y %H:%M:%S").timetuple()))

In [45]:
brs14["Date"] = brs14["Date"].apply(d2s)
brs15["Date"] = brs15["Date"].apply(d2s)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [46]:
acc = {}
for i in section_id_info:
    acc[i] = []

In [47]:
def geo_distance((lat1, lon1), (lat2, lon2)):
    """Calculate geo-distance between 2 points (in meters)."""
    # approximate radius of earth in meters
    R = 6373000.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [56]:
for i, r in brs14.iterrows():
#     print i
    d = r["Date"]
    dd = idloc.apply(geo_distance, args=(r.loco[::-1],))
    if dd.min() <= 1000:
        acc[dd.idxmin()].append(d)

In [57]:
for i, r in brs15.iterrows():
#     print i
    d = r["Date"]
    dd = idloc.apply(geo_distance, args=(r.loco[::-1],))
    if dd.min() <= 1000:
        acc[dd.idxmin()].append(d)

In [60]:
with open("../bdc/acc_time_by_location.pickle", "w") as p:
    pickle.dump(acc, p)